## Paper Review

In [2]:
import tensorflow as tf
import numpy as np
#import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import cv2
from numpy import random as rng
from sklearn.utils import shuffle
import pickle
import time

from tensorflow.keras.layers import Input, Lambda, Conv2D, MaxPooling2D, BatchNormalization, Dense, Flatten, Activation, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import backend as K

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.11.1


---

### Preprocessing the data
Here we load the image data from the downloaded omniglot dataset

In [4]:
def load_images(path):
    #Load the image file and return the coordinates of pixels in the binary image
    
    X = []
    y = []
    lang_dict = {} #used to map the alphabet characters to their class numbers.
    classNum = 0
    
    #Next, we iterate over all the alphabet folders in the Omniglot dataset
    for alphabet in tqdm(sorted(os.listdir(path))):
        lang_dict[alphabet] = [classNum, None]
        #set the path to the current alphabet folder.
        alpha_path = os.path.join(path, alphabet)
        
        #We iterate over over all the letter folders in the current alphabet folder
        for letter in sorted(os.listdir(alpha_path)):
            cat_images = []  #concatenate
            
            #iterates over all the image files in the current letter folder
            for img in sorted(os.listdir(os.path.join(alpha_path, letter))):
                #define the path to the current image file
                img_path = os.path.join(alpha_path, letter, img)
                              
                #read the current image file and convert it to grayscale
                cat_images.append(cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2GRAY))
                              
                y.append(classNum)
            
            classNum+=1
            X.append(cat_images)   #appends the list of images for the current letter
            lang_dict[alphabet][1] = classNum - 1 #Sets the second val in the list to the current class number-1
    #Make X and y numpy arrays
    X = np.array(X)
    print(X.shape)
    y = np.array(y)
    return X, y, lang_dict


    

In [5]:
#We store the path to our training directory and evaluation directory
img_train_PATH = '/Users/siddharthsinha/Desktop/Spring_2023/CSE_5819/Honors_work/omniglot/python/images_background'
img_eval_PATH = '/Users/siddharthsinha/Desktop/Spring_2023/CSE_5819/Honors_work/omniglot/python/images_evaluation'

In [6]:
train_images, train_labels, lang_dict = load_images(img_train_PATH)
val_images, val_labels, lang_dictVal = load_images(img_eval_PATH)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:09<00:00,  3.33it/s]


(964, 20, 105, 105)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:06<00:00,  2.88it/s]

(659, 20, 105, 105)


---
The model can be trained using a batch generator that randomly samples pairs of images from the training set. We also define the functions make_one_shot_task and test_one_shot to test the siamese network. 

In [7]:
def get_batch(batch_size,data_set='train'):
    """ retrieves a batch of data from the specified subset"""
    if data_set == 'train':
        X = train_images
    else:
        X = val_images
    n_classes, n_examples, w, h = X.shape
    cat = rng.choice(n_classes, size=batch_size, replace=False)
    targets = np.zeros((batch_size,))
    targets[batch_size//2:] = 1
    pairs = [np.zeros((batch_size,w,h,1)) for _ in range(2)]
    for i in range(batch_size):
        ex_no = rng.randint(n_examples)
        pairs[0][i,:,:,:] = X[cat[i],ex_no,:,:].reshape(w,h,1)
        cat2 = 0
        if i >= batch_size // 2:
            cat2 = cat[i]
        else:
            cat2 = (cat[i] + rng.randint(1,n_classes)) % n_classes
        ex_no2 = rng.randint(n_examples)
        pairs[1][i,:,:,:] = X[cat2,ex_no2,:,:].reshape(w,h,1)
    return pairs,targets

In [8]:
def make_one_shot_task(N,dset='val'):
    if dset == 'train':
        X = train_images
    else:
        X = val_images
    n_classes, n_examples, w, h = X.shape
    cats = rng.choice(n_classes,size=(N,))
    indices = rng.choice(n_examples,size=(N,))
    true_cat = cats[0]
    ex1 = rng.randint(n_examples)
    test_image = np.array([X[true_cat,ex1]]*N).reshape(N,w,h,1)
    support_set = X[cats,indices].reshape(N,w,h,1)
    targets = np.zeros((N,))
    targets[0] = 1
    
    test_image,support_set,targets = shuffle(test_image,support_set,targets)
    
    return [test_image,support_set], targets

In [9]:
def test_one_shot(model,N,k,dset='val'):
    n_correct = 0
    for _ in range(k):
        inputs, outputs = make_one_shot_task(N,dset)
        preds = model.predict(inputs)
        if np.argmax(outputs) == np.argmax(preds):
            n_correct += 1
    return n_correct / k

---
### Siamese Neural Network

In [10]:
import tensorflow as tf

def get_siamese(input_shape):
    left_input = tf.keras.Input(input_shape)
    right_input = tf.keras.Input(input_shape)

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(64, (10, 10), activation='relu', input_shape=input_shape))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.25))

    model.add(tf.keras.layers.Conv2D(128, (7, 7), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.25))

    model.add(tf.keras.layers.Conv2D(128, (4, 4), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.25))

    model.add(tf.keras.layers.Conv2D(256, (4, 4), activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(4096, activation='sigmoid'))

    left_emb = model(left_input)
    right_emb = model(right_input)

    L1_layer = tf.keras.layers.Lambda(lambda tensors: tf.keras.backend.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([left_emb, right_emb])

    prediction = tf.keras.layers.Dense(1, activation='sigmoid')(L1_distance)

    siamese_net = tf.keras.Model(inputs=[left_input, right_input], outputs=prediction)

    return siamese_net

### Training Loop

In [16]:
num_iterations = 50
batch_size = 128

evaluateEvery = 1
k = 250
N = 5

n_classes, n_examples, w, h = train_images.shape
print(train_images.shape)

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.05,
    decay_steps=4000,
    decay_rate=0.0001)

opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model = get_siamese((w, h, 1))

model.compile(
    loss='binary_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)

model.summary()

(964, 20, 105, 105)
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 105, 105, 1  0           []                               
                                )]                                                                
                                                                                                  
 input_6 (InputLayer)           [(None, 105, 105, 1  0           []                               
                                )]                                                                
                                                                                                  
 sequential_2 (Sequential)      (None, 512)          5926528     ['input_5[0][0]',                
                                                                  'input

In [17]:
losses = []
trainAcc_list = []
valAcc_list = []
x,y = get_batch(batch_size)

for i in range(0,num_iterations+1):
    x,y = get_batch(batch_size)
    loss = model.train_on_batch(x,y)
    if i % evaluateEvery == 0:
        losses.append(loss[0])
        trainAcc = round(test_one_shot(model,N,k,'train') * 100,2)
        valAcc = round(test_one_shot(model,N,k,'val') * 100,2)
        trainAcc_list.append(trainAcc)
        valAcc_list.append(valAcc)
        print('Iteration',i,'Loss:',loss[0],'Acc:',round(loss[1],2),'',end='')
        print(k,str(N)+'-way train accuracy:', trainAcc,'%, ',end='')
        print(k,str(N)+'-way val accuracy:', valAcc,'%')
        


1/1 [==============================] - 0s 189ms/step


1/1 [==============================] - 0s 211ms/step


1/1 [==============================] - 0s 143ms/step


1/1 [==============================] - 0s 151ms/step
Iteration 0 Loss: 12.64622688293457 Acc: 0.51 250 5-way train accuracy: 26.8 %, 250 5-way val accuracy: 28.4 %
1/1 [==============================] - 0s 210ms/step


1/1 [==============================] - 0s 178ms/step


...

1/1 [==============================] - 0s 207ms/step
Iteration 49 Loss: 9.274346351623535 Acc: 0.7 250 5-way train accuracy: 44.0 %, 250 5-way val accuracy: 48.4 %
1/1 [==============================] - 0s 200ms/step


1/1 [==============================] - 0s 199ms/step


1/1 [==============================] - 0s 200ms/step


1/1 [==============================] - 0s 160ms/step


1/1 [==============================] - 0s 175ms/step
Iteration 50 Loss: 9.21512222290039 Acc: 0.73 250 5-way train accuracy: 60.4 %, 250 5-way val accuracy: 60.8 %


In [18]:
print('Final Validation Accuracy:', round(test_one_shot(model,N,k,'val') * 100,2))

1/1 [==============================] - 0s 168ms/step


1/1 [==============================] - 0s 165ms/step
Final Validation Accuracy: 57.2
